In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageOps

def create_id_card(name, job_title, id_num, email, mobile, photo_path, output_path,validity):
    template_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\Temp ID Students.jpg"
    template = Image.open(template_path)
    draw = ImageDraw.Draw(template)
    font_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\raleway\\Raleway-ExtraBold.ttf"

    
    # Name and job title
    Name_font = ImageFont.truetype(font_path, size=48)
    job_title_font = ImageFont.truetype(font_path, size=28)
    
  
    name_bbox = draw.textbbox((0, 0), f"{name}", font=Name_font)
    x_name = (template.width - name_bbox[2]) // 2

    job_title_bbox = draw.textbbox((0, 0), f"{job_title}", font=job_title_font)
    x_job = (template.width - job_title_bbox[2]) // 2

    y_coordinate = 690

    draw.text((x_name, y_coordinate), f"{name}", font=Name_font, fill="black")
    draw.text((x_job, y_coordinate + 52), f"{job_title}", font=job_title_font, fill="grey")



    #Other details
    # Define different font sizes for each type of text
    detail_font = ImageFont.truetype(font_path, size=27)
    
    
    id_num_font = ImageFont.truetype(font_path, size=28)
    email_font = ImageFont.truetype(font_path, size=25)
    mobile_font = ImageFont.truetype(font_path, size=31)

    x = 55
    y = 821

    draw.text((x, y), f"ID NO:", font=detail_font, fill="grey")
    draw.text((x, y + 39), f"Email:", font=detail_font, fill="grey")
    draw.text((x, y + 75), f"Mob   :", font=detail_font, fill="grey")
    
    
    
    draw.text((x + 89, y), f"{id_num}", font=id_num_font, fill="grey")   # Adjust x coordinate for id no.
    draw.text((x + 89, y + 39), f"{email}", font=email_font, fill="grey")    # Adjust x coordinate for email address
    draw.text((x + 90, y + 71), f"{mobile}", font=mobile_font, fill="grey")   # Adjust x coordinate for mobile num

    
    valid_upto = ImageFont.truetype(font_path, size=20)
    draw.text((95, 950), f"Valid to : {validity}", font=detail_font, fill="grey")
    
    # Create a larger circular mask for the photo
    photo = Image.open(photo_path).convert("RGBA")
    mask_size = (photo.size[0] + 55, photo.size[1] + 55) 
    mask = Image.new("L", mask_size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.ellipse((0, 0, mask_size[0], mask_size[1]), fill=255)

    
    
    # Apply the circular mask to the photo
    photo = ImageOps.fit(photo, mask.size, centering=(0.5, 0.5))
    photo.putalpha(mask)

    
    
    # Adjust the coordinates for pasting the circular photo based on your template layout
    template.paste(photo, (149, 290), photo)
    
    
    # Generate barcode
    barcode_output_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\barcode.png"
    generate_barcode(id_num, barcode_output_path)

    # Paste barcode onto the ID card
    barcode = Image.open(barcode_output_path)
    template.paste(barcode, (55, 920))
    

    template.save(output_path)
    template.show()





In [2]:
# Generate a Code 128 barcode for the provided ID number

from barcode import generate
from barcode.writer import ImageWriter

def generate_barcode(id_num, output_path):
    barcode = generate('Code128', f'{id_num}', writer=ImageWriter())
    barcode.save(output_path)


In [4]:
create_id_card(
    name="Abhisekh S Nair",
    job_title="Junior Penetration Tester",
    id_num="E0019",
    email="info@eduzell.com",
    mobile="+91 7592861101",
    photo_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\photos\\1 (Phone).jpg",
    output_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\test_22.png",
    validity= "01-07-2024"
)

TypeError: 'output' cannot be None

In [31]:
def generate_id_cards_from_excel(excel_file, output_folder):
   
    data = pd.read_(excel_file)

    for index, row in data.iterrows():
        create_id_card(
            name=row['Name'],
            job_title=row['Job Title'],
            id_num=row['ID Number'],
            email=row['Email'],
            mobile=row['Mobile'],
            photo_path=row['Photo Path'],
            output_path=f"{output_folder}\\id_card_{index + 1}.png"
        )

In [33]:
import pandas as pd

excel_file_path = "C:/Users/GALDIN/Desktop/Data Science/Project/ID card/Details.csv"
output_folder_path = "C:/Users/GALDIN/Desktop/Data Science/Project/ID card/ID cards"
generate_id_cards_from_excel(excel_file_path, output_folder_path)

In [34]:
!pip install python-barcode


     -------------------------------------- 213.0/213.0 kB 3.3 MB/s eta 0:00:00


In [42]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
from barcode import generate
from barcode.writer import ImageWriter

def create_id_card(name, job_title, id_num, email, mobile, photo_path, output_path):
    template_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\Temp ID Students.jpg"
    template = Image.open(template_path)
    draw = ImageDraw.Draw(template)
    font_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\raleway\\Raleway-ExtraBold.ttf"

    # Name and job title
    Name_font = ImageFont.truetype(font_path, size=48)
    job_title_font = ImageFont.truetype(font_path, size=28)

    name_bbox = draw.textbbox((0, 0), f"{name}", font=Name_font)
    x_name = (template.width - name_bbox[2]) // 2

    job_title_bbox = draw.textbbox((0, 0), f"{job_title}", font=job_title_font)
    x_job = (template.width - job_title_bbox[2]) // 2

    y_coordinate = 690

    draw.text((x_name, y_coordinate), f"{name}", font=Name_font, fill="black")
    draw.text((x_job, y_coordinate + 52), f"{job_title}", font=job_title_font, fill="grey")

    # Other details
    detail_font = ImageFont.truetype(font_path, size=27)

    id_num_font = ImageFont.truetype(font_path, size=28)
    email_font = ImageFont.truetype(font_path, size=25)
    mobile_font = ImageFont.truetype(font_path, size=31)

    x = 55
    y = 821

    draw.text((x, y), f"ID NO:", font=detail_font, fill="grey")
    draw.text((x, y + 39), f"Email:", font=detail_font, fill="grey")
    draw.text((x, y + 75), f"Mob   :", font=detail_font, fill="grey")

    draw.text((x + 89, y), f"{id_num}", font=id_num_font, fill="grey")
    draw.text((x + 89, y + 39), f"{email}", font=email_font, fill="grey")
    draw.text((x + 90, y + 71), f"{mobile}", font=mobile_font, fill="grey")

    # Create a larger circular mask for the photo
    photo = Image.open(photo_path).convert("RGBA")
    mask_size = (photo.size[0] + 55, photo.size[1] + 55)
    mask = Image.new("L", mask_size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.ellipse((0, 0, mask_size[0], mask_size[1]), fill=255)

    # Apply the circular mask to the photo
    photo = ImageOps.fit(photo, mask.size, centering=(0.5, 0.5))
    photo.putalpha(mask)

    # Adjust the coordinates for pasting the circular photo based on your template layout
    template.paste(photo, (149, 290), photo)

    # Generate barcode
    barcode_output_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\barcode.png"
    generate_barcode(id_num, barcode_output_path)

    # Paste barcode onto the ID card
    barcode = Image.open(barcode_output_path)
    template.paste(barcode, (55, 920))

    template.save(output_path)
    template.show()

def generate_barcode(id_num, output_path):
    barcode = generate('Code128', f'{id_num}', writer=ImageWriter())
    barcode.save(output_path)

create_id_card(
    name="Abhisekh S Nair",
    job_title="Junior Penetration Tester",
    id_num="E0019",
    email="info@eduzell.com",
    mobile="+91 7592861101",
    photo_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\photos\\1 (Phone).jpg",
    output_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\test_22.png"
)


TypeError: 'output' cannot be None

In [9]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
from barcode import generate
from barcode.writer import ImageWriter

def create_id_card(name, job_title, id_num, email, mobile, photo_path, output_path,validity):
    template_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\Temp ID Students.jpg"
    template = Image.open(template_path)
    draw = ImageDraw.Draw(template)
    font_path = "C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\raleway\\Raleway-ExtraBold.ttf"

    # Name and job title
    Name_font = ImageFont.truetype(font_path, size=48)
    job_title_font = ImageFont.truetype(font_path, size=28)

    name_bbox = draw.textbbox((0, 0), f"{name}", font=Name_font)
    x_name = (template.width - name_bbox[2]) // 2

    job_title_bbox = draw.textbbox((0, 0), f"{job_title}", font=job_title_font)
    x_job = (template.width - job_title_bbox[2]) // 2

    y_coordinate = 690

    draw.text((x_name, y_coordinate), f"{name}", font=Name_font, fill="black")
    draw.text((x_job, y_coordinate + 52), f"{job_title}", font=job_title_font, fill="#ABB0B8")

    # Other details
    font_path3="C:/Users/GALDIN/Desktop/Data Science/Project/ID card/liberation-serif-font/LiberationSerifBold-vg1L.ttf"
    detail_font = ImageFont.truetype(font_path3, size=27)
    
    
    id_num_font = ImageFont.truetype(font_path3, size=29)
    email_font = ImageFont.truetype(font_path3, size=25)
    mobile_font = ImageFont.truetype(font_path3, size=28)

    x = 55
    y = 790

    draw.text((x, y), f"Badge ID :", font=detail_font, fill="grey")
    draw.text((x, y + 39), f"Email :", font=detail_font, fill="grey")
    draw.text((x, y + 75), f"Mob   :", font=detail_font, fill="grey")

    draw.text((x + 90+40, y), f"{id_num}", font=id_num_font, fill="grey")
    draw.text((x + 91, y + 39), f"{email}", font=email_font, fill="grey")
    draw.text((x + 90, y + 71), f"{mobile}", font=mobile_font, fill="grey")
    
    
    

    # Create a larger circular mask for the photo
    photo = Image.open(photo_path).convert("RGBA")
    mask_size = (photo.size[0] + 55, photo.size[1] + 55)
    mask = Image.new("L", mask_size, 0)
    draw_mask = ImageDraw.Draw(mask)
    draw_mask.ellipse((0, 0, mask_size[0], mask_size[1]), fill=255)

    # Apply the circular mask to the photo
    photo = ImageOps.fit(photo, mask.size, centering=(0.5, 0.5))
    photo.putalpha(mask)

    # Adjust the coordinates for pasting the circular photo based on your template layout
    template.paste(photo, (149, 290), photo)

    # Generate barcode

    barcode_image = generate_barcode(id_num, width=330, height=90)

    # Paste barcode onto the ID card
    template.paste(barcode_image, (55, 902))
    
    font_path3="C:/Users/GALDIN/Desktop/Data Science/Project/ID card/liberation-serif-font/LiberationSerifBold-vg1L.ttf"
    valid_upto = ImageFont.truetype(font_path3, size=20)
    draw.text((121, 988), f"Valid to : {validity}", font=valid_upto, fill="black")

    template.save(output_path)
    template.show()



from barcode import Code128
from barcode.writer import ImageWriter
from PIL import Image
from io import BytesIO

def generate_barcode(id_num, width, height=80):
    barcode = Code128(f'{id_num}', writer=ImageWriter())
    buffer = BytesIO()
    barcode.write(buffer)
    barcode_image = Image.open(buffer)

    # Remove human-readable text by cropping the image
    barcode_image = barcode_image.crop((0, 0, barcode_image.width, barcode_image.height-73))

    # Resize the barcode image
    barcode_image = barcode_image.resize((width, height))

    return barcode_image



create_id_card(
    name="Abhisekh S Nair",
    job_title="Junior Penetration Tester",
    id_num="20231201011",
    email="info@edhjgkfkghfjkyfkgfkuzell.com",
    mobile="+91 7592861101",
    photo_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\photos\\1 (Phone).jpg",
    output_path="C:\\Users\\GALDIN\\Desktop\\Data Science\\Project\\ID card\\test_23.png",
    validity="01-07-2024"
)





In [ ]:
"C:\Users\GALDIN\Desktop\Data Science\Project\ID card\doulos-sil-font\DoulosSil-OwX8.ttf"